In [1]:
import io
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import get_dataset_dir
from collections import defaultdict
import json
from SPARQLWrapper import SPARQLWrapper, JSON
import requests

In [2]:
k = 4
min_movies= 5

In [3]:
def read_item_names():
    file_name = get_dataset_dir() + '/ml-100k/ml-100k/u.item'
    rid_to_name = {}
    with io.open(file_name, 'r', encoding='ISO-8859-1') as f:
        for line in f:
            line = line.split('|')
            rid_to_name[line[0]] = (line[1], line[2])
    return rid_to_name


In [4]:
def get_top_n(predictions, n=5):
    top_n = defaultdict(list)
    for uid, iid, true_r, est, _ in predictions:
        top_n[uid].append((iid, round(est, 3)))
    for uid, user_ratings in top_n.items():
        user_ratings.sort(key=lambda x: x[1], reverse=True)
        top_n[uid] = user_ratings[:n]
    return top_n

In [5]:
def print_to_console(user_id, predictions):
    print('User ID: ', user_id)
    for prediction in predictions:
        print('{:4s} {:<60s} {}'.format(prediction['id'], prediction['name'], prediction['rating']))    

In [6]:
def print_to_file(user_id, predictions):     
    result = {
        'user_id': user_id,
        'predictions': predictions
    }
    with open('result.json', 'w') as outfile:
            json.dump(result, outfile, indent=2);

In [7]:
def load_ides(movies):
    ides = {}
    
    API_ENDPOINT = "https://www.wikidata.org/w/index.php"     
    params = {
        'title' : 'Special%3ASearch', 
        'go' : 'Go',
        'ns0' : '1',
        'ns120' : '1'
    }
        
    for movie in movies:
        params['search'] = movie['name']
        
        res = requests.get(API_ENDPOINT, params=params, allow_redirects=False)

        start_index = res.text.find('''<div class='mw-search-result-heading'><a href="/wiki/''') + len('''<div class='mw-search-result-heading'><a href="/wiki/''')
        end_index = res.text.find('"', start_index)
        
        if start_index != -1 and end_index != -1:
            id = res.text[start_index : end_index]
            ides[movie['name']] = id
    return ides

In [8]:
def request_to_wikidata(id):
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

    spaqrql_query = """
    SELECT ?movie ?publication_date
    WHERE 
    {
        {SELECT ?required_year
        WHERE
        {
          wd:""" + id + """ wdt:P577 ?publication_date.
          BIND(str(YEAR(?publication_date)) AS ?required_year).
        }
        ORDER BY ASC(?required_year) LIMIT 1}

      ?movie wdt:P31 wd:Q11424;
             wdt:P577 ?publication_date.

      BIND(str(YEAR(?publication_date)) AS ?publication_year).
      FILTER (?publication_year = ?required_year).
    }
    """

    sparql.setQuery(spaqrql_query)

    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

In [9]:
def find_movies_with_same_year(id):
    response = request_to_wikidata(id)
    return response['results']['bindings']

In [10]:
user_id = input('ID: ')

data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
sim_options = {
    'name': 'cosine', 
    'user_based': True, 
    'min_support': min_movies
}
algo = KNNWithMeans(k=4, min_k=k, sim_options=sim_options, verbose=True)
algo.fit(trainset)


testset = trainset.build_anti_testset()
testset = list(filter(lambda x: x[0] == user_id, testset))
predictions = algo.test(testset)


top_n = get_top_n(predictions)
top_n = top_n[user_id]
 
movie_names = read_item_names()

predictions = []
for movie_rid, rating in top_n:
    prediction = {}
    prediction['id'] = movie_rid
    prediction['name'] = str(movie_names[movie_rid][0])
    prediction['rating'] = rating
    
    predictions.append(prediction)
    
print_to_console(user_id, predictions)
print_to_file(user_id, predictions)

ides = load_ides(predictions)
print(ides)
print('\n')
for (movie_name, id) in ides.items():
    print('Для', movie_name, 'фильмами с тем же годом выпуска являются:')
    for similar_movie in find_movies_with_same_year(id):
        print('\t', similar_movie['movie']['value'], similar_movie['publication_date']['value'])

ID: 15
Computing the cosine similarity matrix...
Done computing similarity matrix.
User ID:  15
1512 World of Apu, The (Apur Sansar) (1959)                       4.287
853  Braindead (1992)                                             4.285
530  Man Who Would Be King, The (1975)                            4.242
837  Meet John Doe (1941)                                         4.043
198  Nikita (La Femme Nikita) (1990)                              4.041
{'World of Apu, The (Apur Sansar) (1959)': 'Q622376', 'Braindead (1992)': 'Q897429', 'Man Who Would Be King, The (1975)': 'Q1042931', 'Meet John Doe (1941)': 'Q1520732', 'Nikita (La Femme Nikita) (1990)': 'Q743928'}
Для World of Apu, The (Apur Sansar) (1959) фильмами с тем же годом выпуска являются:
	 http://www.wikidata.org/entity/Q190086 1959-03-29T00:00:00Z
	 http://www.wikidata.org/entity/Q190086 1959-09-17T00:00:00Z
	 http://www.wikidata.org/entity/Q74750 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q202548 1959-02-03T00:00:

	 http://www.wikidata.org/entity/Q6045275 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6045545 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6045729 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5989373 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6070419 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6122100 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6047516 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6047619 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6047788 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6047802 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6048196 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6048572 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6048764 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6048793 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6124417 1959-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q6049506 1959-01-01T00

	 http://www.wikidata.org/entity/Q59789 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q63009 1992-02-27T00:00:00Z
	 http://www.wikidata.org/entity/Q53703 1992-06-04T00:00:00Z
	 http://www.wikidata.org/entity/Q32424 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q39969 1992-01-09T00:00:00Z
	 http://www.wikidata.org/entity/Q39970 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q40074 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q26391 1992-01-16T00:00:00Z
	 http://www.wikidata.org/entity/Q26391 1992-09-04T00:00:00Z
	 http://www.wikidata.org/entity/Q27338 1992-01-09T00:00:00Z
	 http://www.wikidata.org/entity/Q72962 1992-09-10T00:00:00Z
	 http://www.wikidata.org/entity/Q72962 1992-10-23T00:00:00Z
	 http://www.wikidata.org/entity/Q19322 1992-06-04T00:00:00Z
	 http://www.wikidata.org/entity/Q24258 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q188384 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q189054 1992-06-16T00:00:00Z
	 http://www.wikidata.

	 http://www.wikidata.org/entity/Q4571676 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4771454 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4705489 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4706039 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4706939 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4837444 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4839311 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4839735 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4661036 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4661120 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4661287 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4779096 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4781243 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4781811 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4661635 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4841166 1992-01-01T00

	 http://www.wikidata.org/entity/Q16430072 1992-12-19T00:00:00Z
	 http://www.wikidata.org/entity/Q16466692 1992-09-03T00:00:00Z
	 http://www.wikidata.org/entity/Q16466693 1992-02-07T00:00:00Z
	 http://www.wikidata.org/entity/Q16466694 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16617261 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16364051 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16372574 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16626139 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16626737 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16676909 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16679478 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16699598 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16631280 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16550612 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16710408 1992-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q166968

...

	 http://www.wikidata.org/entity/Q15143753 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q15632240 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q15792815 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q15651261 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q15815605 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16252271 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16252488 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16252651 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16010850 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16028405 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16253059 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16253391 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16028546 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16248367 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q16254096 1941-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q162490

	 http://www.wikidata.org/entity/Q46044 1990-11-22T00:00:00Z
	 http://www.wikidata.org/entity/Q18442 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18442 1990-01-31T00:00:00Z
	 http://www.wikidata.org/entity/Q18442 1990-05-31T00:00:00Z
	 http://www.wikidata.org/entity/Q20456 1990-11-09T00:00:00Z
	 http://www.wikidata.org/entity/Q3998 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q42047 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q42047 1990-10-11T00:00:00Z
	 http://www.wikidata.org/entity/Q49003 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q49003 1990-07-13T00:00:00Z
	 http://www.wikidata.org/entity/Q49003 1990-10-11T00:00:00Z
	 http://www.wikidata.org/entity/Q82949 1990-11-16T00:00:00Z
	 http://www.wikidata.org/entity/Q82949 1990-12-20T00:00:00Z
	 http://www.wikidata.org/entity/Q82949 1990-12-21T00:00:00Z
	 http://www.wikidata.org/entity/Q94347 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q257630 1990-12-22T00:00:00Z
	 http://www.wikidata.or

	 http://www.wikidata.org/entity/Q984139 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1093200 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1095572 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1192990 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1004410 1990-08-17T00:00:00Z
	 http://www.wikidata.org/entity/Q1004567 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q994716 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1193080 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1166137 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1166137 1990-06-07T00:00:00Z
	 http://www.wikidata.org/entity/Q1166264 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1166264 1990-11-15T00:00:00Z
	 http://www.wikidata.org/entity/Q1169328 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1219960 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1107236 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q1110652 1990-01-01T00:0

	 http://www.wikidata.org/entity/Q4896682 1990-12-15T00:00:00Z
	 http://www.wikidata.org/entity/Q4900000 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5063097 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5002116 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5006125 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5067973 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4853507 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4854541 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4902071 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4754313 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4754918 1990-12-25T00:00:00Z
	 http://www.wikidata.org/entity/Q4762275 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5136241 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5141598 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q5223378 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q4763032 1990-02-01T00

	 http://www.wikidata.org/entity/Q18640182 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q19349784 1990-12-11T00:00:00Z
	 http://www.wikidata.org/entity/Q19359850 1990-05-26T00:00:00Z
	 http://www.wikidata.org/entity/Q19359872 1990-06-23T00:00:00Z
	 http://www.wikidata.org/entity/Q19359890 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809029 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809040 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809052 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809056 1990-01-14T00:00:00Z
	 http://www.wikidata.org/entity/Q18809060 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809061 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809068 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809069 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809071 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q18809072 1990-01-01T00:00:00Z
	 http://www.wikidata.org/entity/Q195707